<a href="https://colab.research.google.com/github/MargareV/VAE-for-de-novo-drug-design/blob/master/Keras_VAE(new).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2019-10-23 18:48:37--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70489432 (67M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  67.22M  90.3MB/s    in 0.7s    

2019-10-23 18:48:38 (90.3 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [70489432/70489432]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.0.1=py37_0
    - ca-certificates==2019.8.28=0
    - certifi==2019.9.11=py37_0
    - cffi==1.12.3=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_

In [12]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint

K.set_image_data_format('channels_first')

print("Image data format: ", K.image_data_format())
print("Image dimension ordering: ", K.image_data_format())
print("Backend: ", K.backend())


Image data format:  channels_first
Image dimension ordering:  channels_first
Backend:  tensorflow


In [6]:
datagen = ImageDataGenerator(zoom_range=0.2)

train_set = datagen.flow_from_directory('/content/drive/My Drive/Images_zinc/train', target_size=(200, 200), class_mode='input', color_mode='grayscale')
validation_set = datagen.flow_from_directory('/content/drive/My Drive/Images_zinc/validation', target_size=(200, 200), class_mode='input', color_mode='grayscale')

Found 12000 images belonging to 1 classes.
Found 3000 images belonging to 1 classes.


In [7]:
# input image dimensions
img_rows, img_cols, img_chns = 200, 200, 1
# number of convolutional filters to use
filters = 64
# convolution kernel size
num_conv = 3

batch_size = 8
if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)
    
latent_dim = 15
intermediate_dim = 1
epsilon_std = 1.0
epochs = 5

print("Original image size: ", original_img_size)

Original image size:  (1, 200, 200)


In [14]:
# encoder network
x = Input(shape=original_img_size)
conv_1 = Conv2D(img_chns,
                kernel_size=(2, 2),
                padding='same', activation='relu')(x)
conv_2 = Conv2D(filters,
                kernel_size=(2, 2),
                padding='same', activation='relu',
                strides=(2, 2))(conv_1)
conv_3 = Conv2D(filters,
                kernel_size=num_conv,
                padding='same', activation='relu',
                strides=1)(conv_2)
conv_4 = Conv2D(filters,
                kernel_size=num_conv,
                padding='same', activation='relu',
                strides=1)(conv_3)
flat = Flatten()(conv_4)
hidden = Dense(intermediate_dim, activation='relu')(flat)

# defined as separate layers because they will be reused later
z_mean = Dense(latent_dim)(hidden)
z_log_var = Dense(latent_dim)(hidden)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# decoder network
decoder_hid = Dense(intermediate_dim, activation='relu')
decoder_upsample = Dense(filters * 100 * 100, activation='relu')

if K.image_data_format() == 'channels_first':
    output_shape = (batch_size, filters, 100, 100)
else:
    output_shape = (batch_size, 100, 100, filters)

print('Output shape 1: ', output_shape)

decoder_reshape = Reshape(output_shape[1:])
decoder_deconv_1 = Conv2DTranspose(filters,
                                   kernel_size=num_conv,
                                   padding='same',
                                   strides=1,
                                   activation='relu')
decoder_deconv_2 = Conv2DTranspose(filters,
                                   kernel_size=num_conv,
                                   padding='same',
                                   strides=1,
                                   activation='relu')

if K.image_data_format() == 'channels_first':
    output_shape = (batch_size, filters, 200, 200)
else:
    output_shape = (batch_size, 200, 200, filters)
    
print('Output shape 2: ', output_shape)

decoder_deconv_3_upsamp = Conv2DTranspose(filters,
                                          kernel_size=(3, 3),
                                          strides=(2, 2),
                                          padding='valid',
                                          activation='relu')
decoder_mean_squash = Conv2D(img_chns,
                             kernel_size=2,
                             padding='valid',
                             activation='sigmoid')

hid_decoded = decoder_hid(z)
up_decoded = decoder_upsample(hid_decoded)
reshape_decoded = decoder_reshape(up_decoded)
deconv_1_decoded = decoder_deconv_1(reshape_decoded)
deconv_2_decoded = decoder_deconv_2(deconv_1_decoded)
x_decoded_relu = decoder_deconv_3_upsamp(deconv_2_decoded)
x_decoded_mean_squash = decoder_mean_squash(x_decoded_relu)

# defining VAE model
vae = Model(x, x_decoded_mean_squash)

def my_vae_loss(y_true, y_pred):
    xent_loss = img_rows * img_cols * metrics.binary_crossentropy(K.flatten(y_true), K.flatten(y_pred))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

vae.compile(optimizer='rmsprop', loss=my_vae_loss, metrics=['accuracy'])
vae.summary()

#callbacks_list = [
#        ModelCheckpoint(final_weights_path, monitor='acc', verbose=1, save_best_only=True),
#        EarlyStopping(monitor='loss', patience=5, verbose=0)
#    ]

Output shape 1:  (8, 64, 100, 100)
Output shape 2:  (8, 64, 200, 200)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 200, 200)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 1, 200, 200)  5           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 64, 100, 100) 320         conv2d_11[0][0]                  
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 64, 100, 100) 36928       conv2d_12[0][0]                  
______________________

In [15]:
history = vae.fit_generator(train_set, shuffle=True, epochs=epochs, steps_per_epoch=batch_size, validation_data=validation_set, validation_steps=50)




Epoch 1/5
8/8 [==============================] - 531s 66s/step - loss: 60096698797885095642398720.0000 - acc: 0.5000 - val_loss: inf - val_acc: 0.4884
Epoch 2/5
8/8 [==============================] - 462s 58s/step - loss: 3120604211430082328351131828224.0000 - acc: 0.5297 - val_loss: inf - val_acc: 0.4886
Epoch 3/5
8/8 [==============================] - 13s 2s/step - loss: 84960755202712387424054411264.0000 - acc: 0.5345 - val_loss: inf - val_acc: 0.5509
Epoch 4/5
8/8 [==============================] - 56s 7s/step - loss: 34436216331958815876683137024.0000 - acc: 0.7868 - val_loss: inf - val_acc: 0.9589
Epoch 5/5
8/8 [==============================] - 90s 11s/step - loss: 19934287318239021770025027753738240.0000 - acc: 0.9715 - val_loss: -160596648.0000 - val_acc: 0.9753
